In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm
import urllib.parse

# ====== 使用者可自訂的參數 ======
base_url = "https://tabelog.com/tokyo/rstLst/soba/"
page_count = 10                 # 要抓幾頁
rating_threshold = 3.5         # 評分篩選門檻
output_csv_name = "tabelog_高評分清單.csv"
# ==================================

headers = {"User-Agent": "Mozilla/5.0"}
df = pd.DataFrame(columns=["店名", "評分", "連結"])

# Step 1: 抓取多頁資料
for page in range(1, page_count + 1):
    print(f"📄 抓取第 {page} 頁...")
    url = f"{base_url}{page}/"
    try:
        res = requests.get(url, headers=headers)
        soup = BeautifulSoup(res.text, "html.parser")
    except Exception as e:
        print(f"⚠️ 第 {page} 頁錯誤：{e}")
        continue

    restaurants = soup.select(".list-rst__wrap")
    for item in restaurants:
        name_tag = item.select_one(".list-rst__rst-name-target")
        rating_tag = item.select_one(".c-rating__val")
        if name_tag and rating_tag:
            try:
                rating = float(rating_tag.text.strip())
            except ValueError:
                continue
            name = name_tag.text.strip()
            link = name_tag['href']
            df.loc[len(df)] = [name, rating, link]
    time.sleep(1)

print(f"\n✅ 共抓取 {len(df)} 筆資料。")

# Step 2: 篩選評分 > 門檻
df_high = df[df["評分"] > rating_threshold].copy()

# Step 3: 抓每家店的地址
df_high["地址"] = None
for idx, row in tqdm(df_high.iterrows(), total=len(df_high), desc="📍抓取地址中"):
    try:
        res = requests.get(row["連結"], headers=headers)
        soup = BeautifulSoup(res.text, "html.parser")
        addr_tag = soup.select_one(".rstinfo-table__address")
        address = addr_tag.text.strip() if addr_tag else "未找到"
    except Exception as e:
        address = f"錯誤：{e}"
    df_high.at[idx, "地址"] = address
    time.sleep(1)

# Step 4: 產生 Google Maps 搜尋網址（用地址）
df_high["Google地圖搜尋"] = df_high["店名"].apply(
    lambda name: "https://www.google.com/maps/search/" + urllib.parse.quote(name)
)

# Step 5: 匯出 CSV
df_high[["店名", "評分", "地址", "Google地圖搜尋"]].to_csv(
    output_csv_name, index=False, encoding="utf-8-sig"
)

print(f"\n✅ 已成功匯出：{output_csv_name}")


📄 抓取第 1 頁...
📄 抓取第 2 頁...
📄 抓取第 3 頁...
📄 抓取第 4 頁...
📄 抓取第 5 頁...
📄 抓取第 6 頁...
📄 抓取第 7 頁...
📄 抓取第 8 頁...
📄 抓取第 9 頁...
📄 抓取第 10 頁...

✅ 共抓取 198 筆資料。


📍抓取地址中: 100%|████████████████████████████████████████████████████████████████████| 19/19 [00:39<00:00,  2.10s/it]


✅ 已成功匯出：tabelog_高評分清單.csv


In [6]:
df_high

,店名,評分,連結,地址,Google地圖搜尋
24,九頭龍蕎麦,3.51,https://tabelog.com/tokyo/A1309/A130905/13110922/,東京都新宿区神楽坂3-3,https://www.google.com/maps/search/%E4%B9%9D%E...
68,肉と蕎麦の店 晴レルヤ,3.58,https://tabelog.com/tokyo/A1319/A131907/13248021/,東京都杉並区松庵3-38-20,https://www.google.com/maps/search/%E8%82%89%E...
77,丹想庵 健次郎,3.58,https://tabelog.com/tokyo/A1311/A131102/13109951/,東京都台東区浅草3-35-3,https://www.google.com/maps/search/%E4%B8%B9%E...
102,そば会席 立会川 吉田家,3.52,https://tabelog.com/tokyo/A1315/A131501/13003333/,東京都品川区東大井2-15-13,https://www.google.com/maps/search/%E3%81%9D%E...
115,馥や,3.63,https://tabelog.com/tokyo/A1302/A130204/13170708/,東京都中央区日本橋小伝馬町14-19 1F・2F,https://www.google.com/maps/search/%E9%A6%A5%E...
118,つきじ 文化人,3.69,https://tabelog.com/tokyo/A1301/A130101/13159030/,東京都中央区築地1-12-16 プレミアム銀座イースト 1F,https://www.google.com/maps/search/%E3%81%A4%E...
138,玉椿,3.52,https://tabelog.com/tokyo/A1311/A131103/13016900/,東京都台東区柳橋1-6-1,https://www.google.com/maps/search/%E7%8E%89%E...
147,鴨の助,3.53,https://tabelog.com/tokyo/A1309/A130902/13269896/,東京都新宿区四谷1-10 第二上野ビル 1F,https://www.google.com/maps/search/%E9%B4%A8%E...
164,松庵,3.72,https://tabelog.com/tokyo/A1309/A130903/13165186/,東京都新宿区荒木町3 北島ビル 1F,https://www.google.com/maps/search/%E6%9D%BE%E...
168,おそばの甲賀,3.74,https://tabelog.com/tokyo/A1307/A130701/13047594/,東京都港区西麻布2-14-5,https://www.google.com/maps/search/%E3%81%8A%E...


In [7]:
df_high["地址"] = None
df_high["午餐價格"] = None
df_high["晚餐價格"] = None

for idx, row in tqdm(df_high.iterrows(), total=len(df_high), desc="📍抓取地址與價格中"):
    try:
        res = requests.get(row["連結"], headers=headers)
        soup = BeautifulSoup(res.text, "html.parser")

        # 地址
        addr_tag = soup.select_one(".rstinfo-table__address")
        address = addr_tag.text.strip() if addr_tag else "未找到"

        # 價格列表（午餐、晚餐）
        price_tags = soup.select(".rdheader-budget__price-target")
        lunch_price = price_tags[0].text.strip() if len(price_tags) > 0 else "未公開"
        dinner_price = price_tags[1].text.strip() if len(price_tags) > 1 else "未公開"

    except Exception as e:
        address = f"錯誤：{e}"
        lunch_price = "錯誤"
        dinner_price = "錯誤"

    df_high.at[idx, "地址"] = address
    df_high.at[idx, "晚餐價格"] = lunch_price
    df_high.at[idx, "午餐價格"] = dinner_price

    time.sleep(1)


📍抓取地址與價格中: 100%|██████████████████████████████████████████████████████████████| 19/19 [00:43<00:00,  2.28s/it]


In [8]:
df_high

,店名,評分,連結,地址,Google地圖搜尋,午餐價格,晚餐價格
24,九頭龍蕎麦,3.51,https://tabelog.com/tokyo/A1309/A130905/13110922/,東京都新宿区神楽坂3-3,https://www.google.com/maps/search/%E4%B9%9D%E...,"￥6,000～￥7,999","￥1,000～￥1,999"
68,肉と蕎麦の店 晴レルヤ,3.58,https://tabelog.com/tokyo/A1319/A131907/13248021/,東京都杉並区松庵3-38-20,https://www.google.com/maps/search/%E8%82%89%E...,"￥6,000～￥7,999",-
77,丹想庵 健次郎,3.58,https://tabelog.com/tokyo/A1311/A131102/13109951/,東京都台東区浅草3-35-3,https://www.google.com/maps/search/%E4%B8%B9%E...,"￥8,000～￥9,999","￥3,000～￥3,999"
102,そば会席 立会川 吉田家,3.52,https://tabelog.com/tokyo/A1315/A131501/13003333/,東京都品川区東大井2-15-13,https://www.google.com/maps/search/%E3%81%9D%E...,"￥15,000～￥19,999","￥5,000～￥5,999"
115,馥や,3.63,https://tabelog.com/tokyo/A1302/A130204/13170708/,東京都中央区日本橋小伝馬町14-19 1F・2F,https://www.google.com/maps/search/%E9%A6%A5%E...,"￥8,000～￥9,999",-
118,つきじ 文化人,3.69,https://tabelog.com/tokyo/A1301/A130101/13159030/,東京都中央区築地1-12-16 プレミアム銀座イースト 1F,https://www.google.com/maps/search/%E3%81%A4%E...,"￥10,000～￥14,999","￥2,000～￥2,999"
138,玉椿,3.52,https://tabelog.com/tokyo/A1311/A131103/13016900/,東京都台東区柳橋1-6-1,https://www.google.com/maps/search/%E7%8E%89%E...,"￥10,000～￥14,999",-
147,鴨の助,3.53,https://tabelog.com/tokyo/A1309/A130902/13269896/,東京都新宿区四谷1-10 第二上野ビル 1F,https://www.google.com/maps/search/%E9%B4%A8%E...,"￥4,000～￥4,999","￥1,000～￥1,999"
164,松庵,3.72,https://tabelog.com/tokyo/A1309/A130903/13165186/,東京都新宿区荒木町3 北島ビル 1F,https://www.google.com/maps/search/%E6%9D%BE%E...,"￥10,000～￥14,999",-
168,おそばの甲賀,3.74,https://tabelog.com/tokyo/A1307/A130701/13047594/,東京都港区西麻布2-14-5,https://www.google.com/maps/search/%E3%81%8A%E...,"￥6,000～￥7,999","￥2,000～￥2,999"


In [10]:
import re

def extract_min_price(price_str):
    """
    從價格字串（如 '￥3,000～￥3,999'）中提取最低價格作為 int
    若無法解析則回傳 None
    """
    if not isinstance(price_str, str):
        return None
    match = re.search(r"￥([\d,]+)", price_str)
    if match:
        return int(match.group(1).replace(",", ""))
    return None

# 對午餐與晚餐欄位都新增最低價格欄
df_high["午餐價格_min"] = df_high["午餐價格"].apply(extract_min_price)
df_high["晚餐價格_min"] = df_high["晚餐價格"].apply(extract_min_price)

# 篩選：只要其中一個低於 5000 就列入
df_cheap = df_high[
    (df_high["午餐價格_min"] < 5000) |
    (df_high["晚餐價格_min"] < 5000)
].copy()

print(f"✅ 共篩選出 {len(df_cheap)} 間午/晚餐便宜的店家")
print(df_cheap[["店名", "午餐價格", "晚餐價格","Google地圖搜尋"]].head())

✅ 共篩選出 12 間午/晚餐便宜的店家
          店名             午餐價格           晚餐價格  \
24     九頭龍蕎麦    ￥6,000～￥7,999  ￥1,000～￥1,999   
77   丹想庵　健次郎    ￥8,000～￥9,999  ￥3,000～￥3,999   
118  つきじ 文化人  ￥10,000～￥14,999  ￥2,000～￥2,999   
147      鴨の助    ￥4,000～￥4,999  ￥1,000～￥1,999   
168   おそばの甲賀    ￥6,000～￥7,999  ￥2,000～￥2,999   

                                            Google地圖搜尋  
24   https://www.google.com/maps/search/%E4%B9%9D%E...  
77   https://www.google.com/maps/search/%E4%B8%B9%E...  
118  https://www.google.com/maps/search/%E3%81%A4%E...  
147  https://www.google.com/maps/search/%E9%B4%A8%E...  
168  https://www.google.com/maps/search/%E3%81%8A%E...  


In [11]:
##儲存高分平價店
df_cheap[["店名", "評分", "地址", "午餐價格", "晚餐價格", "Google地圖搜尋"]].to_csv(
    "tabelog_高評分_平價餐廳.csv", index=False, encoding="utf-8-sig"
)